In [1]:
import utils, os, pandas, glob, logging, random
import numpy as np
from sklearn.model_selection import KFold
import SimpleITK as sitk
# from radiomics import featureextractor
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

# Define basic functions.

In [2]:
def read_dcm_file(dcm_dir):
    """
    Read CT images into a SimpleITK object.
    Parameter:
        A directory of CT images.
    Returns:
        SimpleITK object of the images.
    """
    dcm_files = glob.glob(os.path.join(dcm_dir, '*.dcm'))
    dcm_files.sort(reverse=True)
    dcm_image = sitk.ReadImage(dcm_files)
    return dcm_image

def read_mask_file(ROI_dir):
    """
    Read segmentation mask into a SimpleITK object.
    Parameter: 
        A directory of the mask.
    Returns: 
        SimpleITK object of the mask.
    """
    return sitk.ReadImage(ROI_dir)

def extract_one_sample (dcm_dir, ROI_dir, params):
    """
    Extract radiomics features for each pair of CT scans and the corresponding mask.
    Parameters: 
        dcm_dir: directory of CT images.
        ROI_dir: directory of the mask.
        params: directory of the setting parameters file required for the extraction.
    Returns: 
       radiomics: a dictionary of names and values of radiomics features.
    """
    extractor = featureextractor.RadiomicsFeatureExtractor(params)
    dcm_img = read_dcm_file(dcm_dir)
    ROI = read_mask_file(ROI_dir)
    radiomics = extractor.execute(dcm_img, ROI)
    return radiomics

def get_features_name():
    """
    Get a list of names of radiomics features.
    """
    data2 = utils.load_json(r"E:\UA_radiomics\data_json\11_train_sample.json")
    return list(data2['71057994_CT_ax']['radiomics'].keys())

def get_features_index():
    """
    Set an index to each feature.
    Returns: 
        features_index: a dictionary of names and indices of radiomics features.
    """
    features_name = get_features_name()
    features_index = {}
    for ID in range (len(features_name)):
        features_index[features_name[ID]] = ID
    return features_index

def get_radiomics_label(IDs, data):
    """
    Get radiomics feature values and label of each sample.
    Parameters:
        IDs: list of sample IDs.
        data: dictionary with keys are IDs, and values are information related to those IDs.
    Returns:
        features: a numpy array of radiomics feature values of all the samples in IDs.
        label: a list of samples' label.
    """
    features, label = [],[]
    for ID in IDs:
        label.append(data[ID]['label'])
        features.append(list(data[ID]['radiomics'].values()))
    features = np.asanyarray(features)
    return features, label

# Define 2 functions for ruling out suspected sampling error samples.

In [3]:
def max_HU (dcm_dir, ROI):
    """
    Determine the maximum HU value inside the mask.
    Parameters:
        dcm_dir: directory of CT images.
        ROI: SimpleITK object of the mask.
    Returns:
        maximum_HU: float, the maximum HU value inside the mask.
    """
    dcm = read_dcm_file(dcm_dir)
    dcm = sitk.GetArrayFromImage(dcm)
    stone = np.multiply(dcm, ROI)
    maximum_HU = 0
    for i in range (len(stone)):
        if stone[i].max() > maximum_HU:
            maximum_HU = stone[i].max()
    return float(maximum_HU)

def exclusion(dataset):
    """
    To remove samples whose suspected maxmium HU value (i.e., labeled as pure uric acid but have maxmium HU > 1300).
    Parameter:
        dataset: dictionary contains all pre-screened samples.
    Returns:
        data: dictionary only contains samples that met selection criterion.
    """
    exclude = []
    for k, v in dataset.items():
        for k1, v1 in v.items():
            if modality in k1:
                sample_ID = k1 
                dcm_dir = v1['dcm_dir']
                ROI_dir = v1['ROI_dir']
                ROI = read_mask_file(ROI_dir)
                ROI = sitk.GetArrayFromImage(ROI)[:, :, :, 0]
                if v['label'] == 1:
                    if max_HU (dcm_dir, ROI) > 1300:
                        exclude.append(k)
    data = {}
    for k, v in dataset.items():
        if k not in exclude:
            data[k] = v
    return data

# Define a function for creating the working data dictionary.

In [4]:
def create_data_dict ():
    """
    Create a dictionary with keys are instance IDs, values include label, directory of CT images and the corresponding mask.
    """ 
    #### For 2019 & 2020 cohort ####
    files_19_20 = os.listdir(ROI_path_19_20)
    IDs, data = [], {}
    for file in files_19_20:
        if modality in file:
            filename = file.split('.')[0]
            ID = filename.split('_')[0]
            if '(' in filename:
                dcm_folder = filename [len(ID)+1:-3]
            else:
                dcm_folder = filename [len(ID)+1:]
            dcm_dir = dcm_path_19_20 + ID + '/' + dcm_folder +'/'
            ROI_dir = ROI_path_19_20 + file
            
            if ID not in IDs:
                IDs.append(ID)
                data[ID] = {'label': 1, filename: {'dcm_dir': dcm_dir, 'ROI_dir': ROI_dir}}
            else:
                data[ID][filename] = {'dcm_dir': dcm_dir, 'ROI_dir': ROI_dir}
    #### For 2021 cohort ####
    files_21 = os.listdir(ROI_path_21)
    ROI_file_names = []
    for file in files_21:
        if modality in file:
            ROI_file_names.append(file.split('.')[0])
            
    df = pandas.read_csv(label_file, dtype=str)
    for _, row in df.iterrows():
        patient_id = row['ID']
        if len (patient_id) == 7:
            patient_id = '0' + patient_id
        label = int(row['label'])
        if label == 0:
            label = -1
        data[patient_id] = {'label': label}
        for ROI_file_name in ROI_file_names:
            if ROI_file_name.startswith(patient_id):
                data[patient_id][ROI_file_name] = {'label': label}

    for i in files_21:
        if modality in i:
            filename = i.split('.')[0]
            ID = filename.split('_')[0]
            if '(' in filename:
                dcm_folder = filename[len(ID)+1:-3]
            else:
                dcm_folder = filename [len(ID)+1:]
            dcm_dir = dcm_path_21 + ID + '/' + dcm_folder +'/'
            ROI_dir = ROI_path_21 + i
            data[ID][filename]['dcm_dir'] = dcm_dir
            data[ID][filename]['ROI_dir'] = ROI_dir
    #### remove keys with no samples ####
    data2 = {}
    for k, v in data.items():
        if len(v) > 1:
            data2[k] = v
    return data2

# Define a function for creating splitting data.

In [5]:
def create_datasets (seeding, data):  ### dataset's keys are instance_IDs
    """
    Randomly split data in to test and train data sets. For train set, further split into 5 folds.
    Parameters:
        seeding: int, for reproducibly random split.
        data: dictionary with keys are instances' ID.
    Returns:
        training_sample: dictionary, train dataset with keys are samples' IDs. For each sample ID, there is info indicating whether sub_train or validation sets it belongs to in each fold 
                        (e.g., in Fold_1, it belongs to validation set, to sub_train sets in all the others).
        testing_sample: dictionary, test dataset with keys are samples' IDs
    """
    other, UA = {}, {}
    for k, v in data.items():
        if v['label'] == -1:
            other[k] = v
        if v['label'] == 1:
            UA[k] = v

    UA_IDs    = list (UA.keys())
    other_IDs = list (other.keys())

    random.seed(seeding)
    train_UA_IDs = random.sample(UA_IDs, k = 95)
    test_UA_IDs  = list (set(UA_IDs) - set(train_UA_IDs))

    random.seed(seeding)
    train_others_IDs = random.sample(other_IDs, k = 120)
    test_others_IDs  = list (set(other_IDs) - set(train_others_IDs))

    training_pt, testing_pt = {}, {} # for instances
    for k, v in data.items():
        if k in train_UA_IDs:
            training_pt[k] = v
        if k in train_others_IDs:
            training_pt[k] = v
        if k in test_UA_IDs:
            testing_pt[k] = v
        if k in test_others_IDs:
            testing_pt[k] = v
    
    training_sample, testing_sample = {}, {} # for samples
    for _, v1 in training_pt.items():
        for i in v1.keys():
            if 'ax' in i:
                training_sample[i] = v1[i]
                training_sample[i]['label'] = v1['label']
    for _, v2 in testing_pt.items():
        for j in v2.keys():
            if 'ax' in j:
                testing_sample[j] = v2[j]
                testing_sample[j]['label'] = v2['label']
    
    pure_UA = np.array([k for k, v in training_sample.items() if v['label'] ==  1])
    others  = np.array([k for k, v in training_sample.items() if v['label'] == -1])

    kf =  KFold(n_splits = number_of_folds, random_state = seeding, shuffle = True)

    pure_UA_train, pure_UA_val = [], []
    for train_index, val_index in kf.split(pure_UA):
        pure_UA_train.append(list(pure_UA[train_index]))
        pure_UA_val  .append(list(pure_UA[val_index  ]))

    others_train,  others_val  = [], []
    for train_index, val_index in kf.split(others):
        others_train.append(list(others[train_index]))
        others_val  .append(list(others[val_index  ]))
        
    for i in range (number_of_folds):
        for j in pure_UA_train[i]:
            training_sample[j]['fold_' + str(i)] = 'train'
        for k in others_train[i]:
            training_sample[k]['fold_' + str(i)] = 'train'

        for l in pure_UA_val[i]:
            training_sample[l]['fold_' + str(i)] = 'val'
        for m in others_val[i]:
            training_sample[m]['fold_' + str(i)] = 'val'
    return training_sample, testing_sample

# Define functions for feature selection.

In [23]:
def get_train_val_id_from_fold (fold, data):
    """
    Get sample IDs in train or validation subsets in a particular fold.
    Parameters:
        fold: int, 0-indexing.
        data: dictionary with keys are sample IDs.
    Returns:
        train_ids: list, sample IDs in train subset.
        val_ids: list, sample IDs in validation subset.
    """
    train_ids, val_ids =[], []
    for sample_id, v in data.items():
        if v['fold_'+ str(fold)] == 'train':
            train_ids.append(sample_id)
        if v['fold_'+ str(fold)] == 'val':
            val_ids.append(sample_id)
    return train_ids, val_ids

def train_val_split (ids):
    """
    Randomly divide a set of IDs into 2 parts with the ratio of 8:2.
    Parameter:
        ids: list of IDs.
    Returns:
        train_ids: list of 80% of IDs.
        val_ids: list of 20% of IDs.
    """
    val_size = int(len(ids) * 0.2)
    val_ids = np.random.choice (ids, val_size, replace=False).tolist()
    train_ids = list (set(ids) - set(val_ids))
    return train_ids, val_ids

def select_features (alpha_value, train_data, train_label, val_data, val_label):
    """
    Get features' index and coefficient from LASSO. Only features with non-zero coefficients are filtered.
    Parameters:
        alpha_value: float, regularization value for LASSO.
        train_data: numpy array of radiomics feature values for fitting LASSO.
        train_label: list of labels of train set.
        val_data: numpy array of radiomics feature values for evaluating LASSO.
        val_label: list of labels of validation set.
    Returns:
        ft_idx: numpy array of non-zero features' index 
        ft_coef: numpy array of coefficients.
    """
    lasso = linear_model.Lasso(alpha = alpha_value)
    lasso.fit(train_data, train_label)
    ft_idx  = np.where(lasso.coef_ != 0)[0]
    ft_coef = lasso.coef_[ft_idx]
    return ft_idx, ft_coef

def features_freq_coef (alpha_value, train_ids, data):
    """
    Get accumulated selection times and summation of absolute LASSO coefficient.
    Parameters:
        alpha_value: float, regularization value for LASSO.
        train_ids: list of sample IDs.
    Returns:
        freq: list of feature indices in descending order of accumulated selection times.
        coef: list of feature indices in descending order of summation of absolute coefficients.
    """    
    frequency, coeficient = [], []
    features_name         = get_features_name()
    for ft in features_name:
        frequency .append([ft, 0])
        coeficient.append([ft, 0])
    
    for i in range(100):
        train_subset_ids, val_ids = train_val_split(train_ids)
        
        train_data2, train_label = get_radiomics_label (train_subset_ids, data)
        val_data2  , val_label   = get_radiomics_label (val_ids         , data)
        
        scaler = StandardScaler()
        scaler.fit(train_data2)
        
        train_data = scaler.transform (train_data2)
        val_data   = scaler.transform (val_data2)
        
        ft_idx, ft_coef = select_features (alpha_value, train_data, train_label, val_data, val_label)
        for idx in range (len(ft_idx)):
            frequency  [ft_idx[idx]][1] += 1
            coeficient [ft_idx[idx]][1] += float(abs(ft_coef[idx]))
        
    freq = sorted(frequency,  key = lambda item: item[1], reverse = True)
    coef = sorted(coeficient, key = lambda item: item[1], reverse = True)
    return freq, coef

def most_crucial_fts (number_of_folds, number_features, data, modality, alpha):
    """
    Select the most important features based on their coefficients and selected times.
    Parameters:
        number_of_folds: int, number of folds for cross-validation.
        number_features: int, number of features to be selected.
        data: dictionary, with keys are IDs, and values are information related to those IDs.
        modality: str, the acquisition plane of interest (e.g., CT_ax: axial plane).
        alpha: float, regularization value for LASSO.
    Returns: 
        A list of selected features' indices.
        Note that number of features in this list might differ from number_features due to some unmet selection criteria.
    """
    features = get_features_index()
    for k, v in features.items():
        features[k] = {'index': v, 'freq': 0, 'coef': 0}
    for fold in range (number_of_folds):
        train_ids, _ = get_train_val_id_from_fold (fold, data)        
        freq_fold, coef_fold = features_freq_coef (alpha, train_ids, data)
        for i in range (len(freq_fold)):
            features[freq_fold[i][0]]['freq'] += freq_fold[i][1]
            features[coef_fold[i][0]]['coef'] += coef_fold[i][1]
    df = []
    for k, v in features.items():
        name = k
        index = v['index']
        freq  = v['freq']
        coef  = v['coef']
        df.append ([name, index, freq, coef])
    freq = sorted(df, key = lambda item: item[2], reverse = True)
    coef = sorted(df, key = lambda item: item[3], reverse = True)
    selected = []
    for i in coef[:20]: # selection criteria prioritizes coef
        if i in freq[:20]:
            selected.append (i[1])           
    return selected[:number_features]

def optimal_set_of_features (crucial_features_idx):
    results = []
    for n_ft in range (len(crucial_features_idx)):
        features_index = crucial_features_idx[:n_ft+1]
        acc, sen, spe, AUC = 0, 0, 0, 0
        for fold in range (number_of_folds):
            _, val_acc, _, sensi, speci, aurocc, _ = training (fold, test_data, test_label, features_index, report_on='val')
            acc   += round (val_acc,2)
            sen   += round (sensi,2)
            spe   += round (speci,2)
            AUC   += round (aurocc,2)
        results.append([features_index, acc/5, sen/5, spe/5, AUC/5])
    return results

# Define functions for reporting performance

In [7]:
def report(labels, data, classifier, ft_idx, report_test=False):
    """
    Report performance metrics for a particular data set.
    Parameters:
        labels: list of sample labels.
        data: numpy array of radiomics features values.
        classifier: the model that has been trained.
        ft_idx: list of feature indices in consideration.
    Returns:
        sensi: float, sensitivity where pure uric acid samples are considered as positive case.
        speci: float, specificity.
        auc: float, area under the ROC curve.
    """
    title = list(map(str, ft_idx))
    title = ''.join(str(ft_idx))
    preds = classifier.predict(data)
    probs = classifier.predict_proba(data)
    trans_labels = []
    for label in labels:
        if label == -1:
            trans_labels.append(0)
        else:
            trans_labels.append(1)
    for index in range(len(preds)):
        if preds[index] == -1:
            preds[index] = 0

    acc = utils.cal_simple_acc(preds, trans_labels)
    sensi, speci = utils.cal_sensi_speci(trans_labels, preds)
#     conf_matrix  = confusion_matrix(trans_labels, preds)
#     utils.make_confusion_matrix (conf_matrix, categories= ['Others', 'Pure uric acid'], title=title)
    
    if report_test:
        probs = np.array(probs)
        trans_labels = np.array(trans_labels)
        f1_score, auc = utils.Precision_Recall_curve(preds, trans_labels, probs[:, 1])
#         print (f'Sen: {round(sensi,3)}; Spe: {round(speci,3)}; AUPRC: {round(auc,3)}; F1: {round(f1_score, 3)}')
#         print ('_________________________')
    else: 
        f1_score = 0
        auc = utils.cal_auc(trans_labels, probs, title)
#         utils.visualize_roc (trans_labels, probs, title)
#         print (f'Sen: {round(sensi,3)}; Spe: {round(speci,3)}; AUC: {round(auc,3)}')
#         print ('_________________________')
    return sensi, speci, auc, f1_score

# Define a function for model training.

In [8]:
 def training(fold, test_data, test_label, features_selected_index, report_on='test'):
    """
    Train the model with optimal hyperparameters and selected features.
    Report performance for train, validation or test set by uncommment on corresponding line of code.
    Parameters:
        fold: int, 0-indexing.
        test_data: numpy array of radiomics features values of test set.
        test_label: list of sample labels of test set
        report: str, specify which set reporting performance on. 'test' by default.
    Returns:
        train_acc: float, accuracy on train set.
        val_acc: float, accuracy on validation set.
        test_acc: float, accuracy on test set.
        sensi: float, sensitivity on the specified set.
        speci: float, specificity on the specified set.
        auc: float, area under the ROC curve for train or validation set, under the Precicion-Recall curve for test set.
        f1_score: float, 0 for train or validation set.
    """
    train_ids, val_ids = get_train_val_id_from_fold (fold, train_dataset)   
    train_data, train_label =  get_radiomics_label(train_ids, train_dataset)
    val_data  , val_label   =  get_radiomics_label(val_ids  , train_dataset)
    
    scaler = StandardScaler()
    scaler.fit(train_data)
    train_data = scaler.transform(train_data)
    val_data   = scaler.transform(val_data)
    test_data  = scaler.transform(test_data)
    
    train_data = train_data[:, features_selected_index]
    val_data   = val_data  [:, features_selected_index]
    test_data  = test_data [:, features_selected_index]
    
    classifier = SVC(probability=True, random_state=seed, C=C, gamma=gamma)
    classifier.fit(train_data, train_label)
    
    train_acc = classifier.score(train_data, train_label)
    val_acc   = classifier.score(val_data  , val_label)
    test_acc  = classifier.score(test_data , test_label)

#     print (f'Of fold #{fold}')
#     print (f"Train_acc: {round(train_acc,3)}; Val_acc: {round(val_acc,3)}; Test_acc: {round(test_acc,3)}")
    
    sensi, speci, auc, f1_score = 0, 0, 0, 0
    if report_on == 'train':
        sensi, speci, auc, _ = report(train_label, train_data, classifier, features_selected_index)
    if report_on == 'val':
        sensi, speci, auc, _ = report(val_label, val_data, classifier, features_selected_index)
    if report_on == 'test':
        sensi, speci, auc, f1_score = report(test_label , test_data , classifier, features_selected_index, report_test=True)
    return train_acc, val_acc, test_acc, sensi, speci, auc, f1_score

# Define a function for final model training.

In [9]:
def re_train (train_dataset, test_data, test_label, features_selected_index):
    """
    Re-train with optimal hyperparameters and selected features to obtain the final model.
    Report performance for the test set.
    Parameters:
        train_dataset: dictionary of the entire train set (i.e., the remaing of the original data set after hold-out the test set).
        test_data: numpy array of radiomics features values of test set.
        test_label: list of sample labels of test set
    Returns:
        test_acc: float, accuracy on the test set.
        sensi: float, sensitivity on the test set.
        speci: float, specificity on the test set.
        auc: float, area under the Precicion-Recall curve.
        f1_score: float, F1-score on the test set.
    """
    train_ids = train_dataset.keys()    
    train_data, train_label =  get_radiomics_label(train_ids, train_dataset)
    
    scaler = StandardScaler()
    scaler.fit(train_data)
    train_data = scaler.transform(train_data)
    test_data  = scaler.transform(test_data)
    
    train_data = train_data[:, features_selected_index]
    test_data  = test_data [:, features_selected_index]
    
    classifier = SVC(probability=True, random_state=seed, C=C, gamma=gamma)
    classifier.fit(train_data, train_label)
    
    test_acc  = classifier.score(test_data , test_label)
    sensi, speci, auc, f1_score = report(test_label , test_data , classifier, features_selected_index, report_test=True)
    return test_acc, sensi, speci, auc, f1_score

# Execution

## 1. Set inital parameters.

In [10]:
ROI_path_19_20  = 'D:/Stone_composition/data_19_20/ROI/'  ### This dataset only contains 100UA patients
dcm_path_19_20  = 'D:/Stone_composition/data_19_20/DICOM/'
ROI_path_21  = 'D:/Stone_composition/data_21/ROI/' 
dcm_path_21  = 'D:/Stone_composition/data_21/DICOM/'
label_file   = 'D:/Stone_composition/data_21/label.csv'
params       = 'D:/Stone_composition/radiomics/binh/Params_stone_composition.yaml'
feature_json = 'D:/Stone_composition/radiomics/data/11.json'
modality     = 'CT_ax'

number_of_folds = 5
number_features = 20
seed            = 2021
alpha           = 0.08
C               = 0.4
gamma           = 0.075

## 2. Create working data object.

In [11]:
# data = create_data_dict ()
# data2 = exclusion(data)

## 3. Train-test split data and 5-fold split.

In [12]:
# train_dataset, test_dataset = create_datasets(seed, data2) ### keys are sample_IDs
train_dataset = utils.load_json(r"E:\UA_radiomics\data_json\11_train_sample.json")
test_dataset = utils.load_json(r"E:\UA_radiomics\data_json\11_test_sample.json")

In [13]:
test_ids = test_dataset.keys()
test_data, test_label =  get_radiomics_label(test_ids, test_dataset)

## 4. Extract and update the working data with radiomics features.

In [14]:
# for _, v1 in data2.items():
#     for k, v in v1.items():
#         if 'ax' in k:
#             dcm_dir = v['dcm_dir']
#             ROI_dir = v['ROI_dir']
#             radiomics_raw = extract_one_sample (dcm_dir, ROI_dir, params)
#             radiomics_filtered = {}
#             for key, value in radiomics_raw.items():
#                 if key[:5] in ['origi', 'wavel']: # only consider original and wavelet features
#                     radiomics_filtered[key] = float(value)
#             v['radiomics'] = radiomics_filtered

## 5. Feature selection

In [15]:
features_selected_index = most_crucial_fts (number_of_folds, number_features, train_dataset, modality, alpha)
# features_selected_index = [20, 28, 214, 420, 605, 544, 323, 330, 233,  395]  ### best performance
print (f"Selected features: {features_selected_index}")
features_name = get_features_name()
for i in features_selected_index:
    print (features_name[i])

Selected features: [20, 28, 214, 233, 420, 605, 544, 39, 323, 330, 46, 38, 173, 395, 209, 117, 157, 549, 140]
original_firstorder_Maximum
original_firstorder_Skewness
wavelet-LHL_firstorder_Skewness
wavelet-LHL_glcm_InverseVariance
wavelet-HLL_glcm_JointAverage
wavelet-HHL_glcm_InverseVariance
wavelet-HLH_glszm_LowGrayLevelZoneEmphasis
original_glcm_DifferenceEntropy
wavelet-LHH_glcm_Idn
wavelet-LHH_glcm_MCC
original_glcm_Imc2
original_glcm_DifferenceAverage
wavelet-LLH_glszm_SizeZoneNonUniformity
wavelet-HLL_firstorder_Median
wavelet-LHL_firstorder_Median
wavelet-LLH_firstorder_Minimum
wavelet-LLH_glrlm_RunEntropy
wavelet-HLH_glszm_SmallAreaLowGrayLevelEmphasis
wavelet-LLH_glcm_InverseVariance


In [24]:
find_set_of_features = optimal_set_of_features (features_selected_index)

Of fold #0
Train_acc: 0.863; Val_acc: 0.94; Test_acc: 0.871
Of fold #1
Train_acc: 0.894; Val_acc: 0.819; Test_acc: 0.865
Of fold #2
Train_acc: 0.876; Val_acc: 0.902; Test_acc: 0.871
Of fold #3
Train_acc: 0.888; Val_acc: 0.854; Test_acc: 0.871
Of fold #4
Train_acc: 0.885; Val_acc: 0.902; Test_acc: 0.877
Of fold #0
Train_acc: 0.912; Val_acc: 0.94; Test_acc: 0.92
Of fold #1
Train_acc: 0.915; Val_acc: 0.892; Test_acc: 0.92
Of fold #2
Train_acc: 0.909; Val_acc: 0.939; Test_acc: 0.92
Of fold #3
Train_acc: 0.924; Val_acc: 0.89; Test_acc: 0.92
Of fold #4
Train_acc: 0.909; Val_acc: 0.927; Test_acc: 0.92
Of fold #0
Train_acc: 0.909; Val_acc: 0.952; Test_acc: 0.926
Of fold #1
Train_acc: 0.918; Val_acc: 0.88; Test_acc: 0.926
Of fold #2
Train_acc: 0.921; Val_acc: 0.902; Test_acc: 0.926
Of fold #3
Train_acc: 0.921; Val_acc: 0.878; Test_acc: 0.933
Of fold #4
Train_acc: 0.906; Val_acc: 0.939; Test_acc: 0.926
Of fold #0
Train_acc: 0.921; Val_acc: 0.94; Test_acc: 0.933
Of fold #1
Train_acc: 0.933; Val_a

In [25]:
find_set_of_features

[[[20], 0.882, 0.946, 0.818, 0.9359999999999999],
 [[20, 28], 0.9179999999999999, 0.96, 0.874, 0.95],
 [[20, 28, 214], 0.9099999999999999, 0.95, 0.868, 0.958],
 [[20, 28, 214, 233], 0.9199999999999999, 0.95, 0.8880000000000001, 0.962],
 [[20, 28, 214, 233, 420],
  0.9219999999999999,
  0.9560000000000001,
  0.8879999999999999,
  0.966],
 [[20, 28, 214, 233, 420, 605], 0.924, 0.952, 0.898, 0.966],
 [[20, 28, 214, 233, 420, 605, 544],
  0.9339999999999999,
  0.968,
  0.9039999999999999,
  0.968],
 [[20, 28, 214, 233, 420, 605, 544, 39],
  0.9339999999999999,
  0.9620000000000001,
  0.908,
  0.9700000000000001],
 [[20, 28, 214, 233, 420, 605, 544, 39, 323],
  0.9299999999999999,
  0.962,
  0.898,
  0.9720000000000001],
 [[20, 28, 214, 233, 420, 605, 544, 39, 323, 330],
  0.924,
  0.952,
  0.898,
  0.9720000000000001],
 [[20, 28, 214, 233, 420, 605, 544, 39, 323, 330, 46],
  0.924,
  0.952,
  0.898,
  0.9720000000000001],
 [[20, 28, 214, 233, 420, 605, 544, 39, 323, 330, 46, 38],
  0.92799

## 6. Train and evaluate by 5-fold cross validation.

In [17]:
train, val, sen, spe, AUPRC = 0, 0, 0, 0, 0
for fold in range (number_of_folds):
    train_acc, val_acc, test_acc, sensi, speci, auprc, f1_score = training (fold, test_data, test_label, features_selected_index, report_on='val')
    train += train_acc
    val   += val_acc
    sen   += sensi
    spe   += speci
    AUPRC += auprc
print (f"Mean train_acc: {round((train/5),2)}, val_acc: {round((val/5),2)}, val_sen: {round((sen/5),2)}, val_spe: {round((spe/5),2)}, val_AUC: {round((AUPRC/5),2)}")

Of fold #0
Train_acc: 0.945; Val_acc: 0.929; Test_acc: 0.933
Of fold #1
Train_acc: 0.958; Val_acc: 0.916; Test_acc: 0.92
Of fold #2
Train_acc: 0.952; Val_acc: 0.963; Test_acc: 0.939
Of fold #3
Train_acc: 0.955; Val_acc: 0.89; Test_acc: 0.926
Of fold #4
Train_acc: 0.946; Val_acc: 0.951; Test_acc: 0.92
Mean train_acc: 0.95, val_acc: 0.93, val_sen: 0.95, val_spe: 0.91, val_AUC: 0.97


## 7. Performance of the final model on the test set.

In [19]:
test_acc, sensi, speci, auc, f1_score = re_train (train_dataset, test_data, test_label, features_selected_index)

f1=0.874 AUPRC=0.955
